Common Types have "character"; how they implement different interfaces (behaviors) which enables them to use them in interesting ways

- https://learn.microsoft.com/en-us/dotnet/api/system.int32?view=netframework-4.8

# Value and Reference Types

- Passing By Value and By Reference

In [ ]:
class Shape
{
     public int X; public int Y;

     public override string ToString()
     {
        return $"X: {X}, Y: {Y}";
     }
}

void increment(ref int x) // passing value types by reference
{
    x++;
}

void move(Shape shape, int x, int y)
{
    shape.X += x;
    shape.Y += y;
}

int x = 5;
var square = new Shape() { X = 5, Y = 5 };

void print()
{
    Console.WriteLine($"x: {x}");
    Console.WriteLine($"shape: {square}");
}

print();
increment(ref x);
move(square, 3, 3);
print();

Links

- https://learn.microsoft.com/en-us/dotnet/api/system.int32?view=net-8.0; with each new release common types are implementing more and more behaviors
- https://learn.microsoft.com/en-us/dotnet/csharp/fundamentals/types/
- https://learn.microsoft.com/en-us/dotnet/csharp/programming-guide/events/how-to-raise-base-class-events-in-derived-classes
- https://source.dot.net/#System.Private.CoreLib/src/libraries/System.Private.CoreLib/src/System/Reflection/ConstructorInvoker.cs,309; just to scare you guys; we can go deep

*The Type System and Base Class Library are done tastefully*

In [ ]:
// IDisposable
// https://learn.microsoft.com/en-us/dotnet/api/system.idisposable

class HeavyResource: IDisposable
{
    string data = "someLargeLookupThing";

    public void SomeOperation() { }

    public void Dispose()
    {
        Console.WriteLine("Disposing resources");
        data = null;
    }
}

using(var resource = new HeavyResource())
{
    // do something with heavy resource
}

- https://learn.microsoft.com/en-us/dotnet/api/system.collections.ienumerable

IEnumerable plays a central role in all things collections; LINQ takes it a one step further

In [ ]:
// yield

IEnumerable<int> GenerateDate()
{
    yield return 1;
    yield return 2;
    yield return 3;
}

foreach(var d in GenerateDate())
    Console.WriteLine(d);

In [ ]:
// anonymous type

var data = new { Key = 1, Value = "Something" };
data.Key // its anonymous but still "typed"

Convinced? C# and other OOP languages are not always Java Clones; they share the OOP foundations and may look familiar because of C family language

![Java Clone?](images/java-clones.jpg)

# Extension Methods

In [ ]:
//Extension Methods

abstract class Shape
{
    public int X;
    public int Y;
}

class Square : Shape
{
    public int Side { get; set; }
}

static void Move(this Shape shape, int x, int y) // class, interface
{
    shape.X += x;
    shape.Y += y;
}

var square = new Square() { X = 3, Y = 3, Side = 5 };
square.Move(3, 4);

// but why?

# LINQ

In [ ]:
using System.Linq;

abstract class Person
{
    public string FirstName { get; set; }
    public string LastName { get; set; }

    public override string ToString()
    {
        return $"{FirstName} {LastName}";
    }
}

class Teacher : Person
{
    public string Subject { get; set; }

    public override string ToString()
    {
        return $"{base.ToString()}, Subject: {Subject}";
    }
}

var teachers = new Teacher[]
{
    new Teacher() { FirstName = "Teacher", LastName = "A", Subject = "Maths" },
    new Teacher() { FirstName = "Teacher", LastName = "B", Subject = "Physics" },
    new Teacher() { FirstName = "Teacher", LastName = "C", Subject = "Biology" },
    new Teacher() { FirstName = "Teacher", LastName = "D", Subject = "Chemistry" }
};

var subjects = new[] { "Maths", "Physics" };

var q = from t in teachers
        where subjects.Contains(t.Subject)
        select t;

var results = q.ToArray();
var q2 = results.OrderByDescending(s => s.LastName);

foreach(var r in q2)
    Console.WriteLine(r);

In [ ]:
// projection
var q3 = from r in results
         select new { r.LastName, r.Subject };
foreach(var r in q3)
    Console.WriteLine($"{r.LastName} teaches {r.Subject}");

In [ ]:
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer, *-*"

# SQL Server Introduction

In [ ]:
using Microsoft.EntityFrameworkCore;


class TeamMember
{
    public int TeamMemberId { get; set; }
    public string TeamMemberName { get; set; } = string.Empty;
    public string TeamMemberEmail { get; set; } = string.Empty;
    public bool Concerned { get; set; } = false;
}

class JiraDbContext : DbContext
{
    public DbSet<TeamMember> TeamMembers { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        var connectionString = "Server=.;Database=JiraDb;Trusted_Connection=True;TrustServerCertificate=True;";
        optionsBuilder.UseSqlServer(connectionString);
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        //base.OnModelCreating(modelBuilder);
        modelBuilder.Entity<TeamMember>(entity =>
        {
            entity.HasIndex(e => e.TeamMemberEmail).IsUnique();
        });
    }
}

In [ ]:
using(var db = new JiraDbContext())
{
    db.Database.EnsureCreated();

    db.TeamMembers.Add(new TeamMember()
    {
            TeamMemberEmail = "khurram.aziz@uworx.co.uk", Concerned = true,
            TeamMemberName = "Khurram Aziz"
    });
    db.SaveChanges();

    var q = from m in db.TeamMembers
            select m;
        
    foreach(var r in q)
        Console.WriteLine($"{r.TeamMemberId} - {r.TeamMemberEmail} - {r.TeamMemberName}");
}

# Oracle?

![Oracle?](images/stay-away.jpg)